# Keep track of tasking deliveries

Time between task and acquisition

Time between acquisition and delivery

In [1]:
import capella_console_client
import geopandas as gpd
import pandas as pd
import json

In [2]:
with open('/home/jovyan/capella/credentials.json') as f: # USER: Input path to credentials.json file.
    data = json.load(f)
    username = data['username']
    password = data['password']

client =  capella_console_client.CapellaConsoleClient(username, password)

In [3]:
gf = gpd.GeoDataFrame.from_features(client.list_tasking_requests())
len(gf)

39

In [4]:
# Remove cancelled orders
gf['final_status'] = gf['statusHistory'].apply(lambda x: x[0]['code']) #final_status
gf = gf[gf.final_status == 'completed']
len(gf)

14

In [5]:
# Consider only 1_day tier
gf = gf[gf.collectionTier == '1_day']
len(gf)

13

In [6]:
task_history = pd.DataFrame(gf.iloc[0].statusHistory)
task_history

,time,code,message
0,2023-10-06T12:31:33.696000,completed,Tasking request has been completed.
1,2023-10-06T01:32:17.797000,accepted,Request can be completely satisfied during val...
2,2023-10-06T01:29:10.611000,submitted,Request approved and submitted for scheduling
3,2023-10-06T00:45:03.709000,review,Tasking request ready for review.
4,2023-10-06T00:45:01.576000,received,Request created


In [7]:
def time2delivery(row):
    df = pd.DataFrame(row.statusHistory)
    df['time'] = pd.to_datetime(df.time)
    dt = df[df.code == 'completed'].iloc[0].time - df[df.code == 'submitted'].iloc[0].time
    dt = dt / pd.Timedelta(hours=1)
    return dt

In [8]:
time2delivery(gf.iloc[0])

11.039745833333333

In [9]:
print('Submission to delivery (hrs)')
gf.apply(time2delivery, axis=1).describe().apply('{:,.1f}'.format)

Submission to delivery (hrs)


count    13.0
mean     24.0
std      19.9
min       4.7
25%      11.0
50%      18.3
75%      26.8
max      73.8
dtype: object

In [10]:
#? which task took 73 hours?
gf['time2deliver'] = gf.apply(time2delivery, axis=1)
cols = ['time2deliver','taskingrequestName','taskingrequestDescription']
gf[cols]

,time2deliver,taskingrequestName,taskingrequestDescription
0,11.039746,Paradise: subdaily-9,Paradise subdaily test
2,11.395361,Paradise: subdaily-10,Paradise subdaily test
7,26.779765,Paradise: subdaily-6,Paradise subdaily test
10,18.334758,Paradise Snotel 45 Retask Oct2,snotel station spotlight (ID: b3cc101b-ebe0-4d...
11,73.809439,Baker Field Work: 3,Easton glacier
12,43.701188,Baker Field Work: 2,Easton glacier
13,10.944945,Baker Field Work: 1,Easton glacier
14,47.211687,Rainier 53 NE Strip Retask,"camp muir, NE strike (ID: 033fd86f-9271-42d9-9..."
15,12.848820,Paradise Snotel 45 Retask,snotel station spotlight (ID: b3cc101b-ebe0-4d...
18,23.174859,Rainier 53 NE Strip,"camp muir, NE strike"


In [11]:
# Doesn't quite work for repeat tasks like Baker field work.

In [12]:
#gf.windowDuration should be 86400 for 1_day

In [13]:
gf.iloc[0]

geometry                                  POINT Z (-121.744564 46.796787 1589)
submissionTime                                      2023-10-06T00:45:01.575000
taskingrequestId                          89c04f42-e6df-4873-82d3-9c37cf5dc480
taskingrequestName                                        Paradise: subdaily-9
taskingrequestDescription                               Paradise subdaily test
taskingrequestType                                                      custom
orgId                                     e075d057-19cf-4bfb-9b07-9fee713bb05e
userId                                    0653314d-010a-423b-8a8e-11ab48983152
repeatrequestId                                                           None
windowOpen                                                 2023-10-05T21:18:02
windowDuration                                                         86401.0
windowClose                                                2023-10-06T21:18:03
collectionTier                                      

In [14]:
# Filter for windowOpen within 24 hours of task submission
def submission2open(row):
    dt = pd.to_datetime(row.windowOpen) - pd.to_datetime(row.submissionTime) 
    dt = dt / pd.Timedelta(hours=1)
    return dt

gf['submission2windowOpen'] = gf.apply(submission2open, axis=1)
cols = ['submission2windowOpen','time2deliver','taskingrequestName','taskingrequestDescription']
gf[cols]

,submission2windowOpen,time2deliver,taskingrequestName,taskingrequestDescription
0,-3.449882,11.039746,Paradise: subdaily-9,Paradise subdaily test
2,-3.310301,11.395361,Paradise: subdaily-10,Paradise subdaily test
7,10.587220,26.779765,Paradise: subdaily-6,Paradise subdaily test
10,-0.022800,18.334758,Paradise Snotel 45 Retask Oct2,snotel station spotlight (ID: b3cc101b-ebe0-4d...
11,47.626122,73.809439,Baker Field Work: 3,Easton glacier
12,23.627516,43.701188,Baker Field Work: 2,Easton glacier
13,-0.047348,10.944945,Baker Field Work: 1,Easton glacier
14,1.323689,47.211687,Rainier 53 NE Strip Retask,"camp muir, NE strike (ID: 033fd86f-9271-42d9-9..."
15,-0.039153,12.848820,Paradise Snotel 45 Retask,snotel station spotlight (ID: b3cc101b-ebe0-4d...
18,-0.037969,23.174859,Rainier 53 NE Strip,"camp muir, NE strike"


In [15]:
def acquisition2delivery(row):
    ''' time between satellite collection and imagery delivery '''
    # Slower b/c must go into collects
    collects = client.get_collects_for_task(row.taskingrequestId)
    c = collects[0] #always only 1?
    df = pd.DataFrame(c['collectStatusHistory'])
    
    df['time'] = pd.to_datetime(df.time)
    dt = df[df.code == 'delivered'].iloc[0].time - df[df.code == 'collected'].iloc[0].time
    dt = dt / pd.Timedelta(hours=1)
    return dt

In [16]:
acquisition2delivery(gf.iloc[0])

1.4127166666666666

In [17]:
gf['acquisition2delivery'] = gf.apply(acquisition2delivery, axis=1)

In [18]:
cols = ['acquisition2delivery','submission2windowOpen','time2deliver','taskingrequestName','taskingrequestDescription']
gf[cols]

,acquisition2delivery,submission2windowOpen,time2deliver,taskingrequestName,taskingrequestDescription
0,1.412717,-3.449882,11.039746,Paradise: subdaily-9,Paradise subdaily test
2,2.084954,-3.310301,11.395361,Paradise: subdaily-10,Paradise subdaily test
7,3.436076,10.587220,26.779765,Paradise: subdaily-6,Paradise subdaily test
10,1.303516,-0.022800,18.334758,Paradise Snotel 45 Retask Oct2,snotel station spotlight (ID: b3cc101b-ebe0-4d...
11,8.238487,47.626122,73.809439,Baker Field Work: 3,Easton glacier
12,2.565781,23.627516,43.701188,Baker Field Work: 2,Easton glacier
13,2.780717,-0.047348,10.944945,Baker Field Work: 1,Easton glacier
14,20.261465,1.323689,47.211687,Rainier 53 NE Strip Retask,"camp muir, NE strike (ID: 033fd86f-9271-42d9-9..."
15,2.316606,-0.039153,12.848820,Paradise Snotel 45 Retask,snotel station spotlight (ID: b3cc101b-ebe0-4d...
18,2.042424,-0.037969,23.174859,Rainier 53 NE Strip,"camp muir, NE strike"


In [19]:
# NOTE: polar orbits probably have longer delivery time due to downlink station distribution

In [20]:
# What is going on with the re-task?
gf.loc[34]

geometry                                           POINT Z (-121.733 46.842 0)
submissionTime                                      2023-08-14T16:57:54.083000
taskingrequestId                          dad725b7-d775-4cc2-b05e-7a94ac238140
taskingrequestName                          Mt Rainier Gibraltar Rock 2 Retask
taskingrequestDescription    Mt Rainier Gibraltar, orbit=97, pol=VV, desc (...
taskingrequestType                                                      custom
orgId                                     e075d057-19cf-4bfb-9b07-9fee713bb05e
userId                                    0653314d-010a-423b-8a8e-11ab48983152
repeatrequestId                                                           None
windowOpen                                          2023-08-14T16:54:38.757000
windowDuration                                                       90321.243
windowClose                                                2023-08-15T18:00:00
collectionTier                                      

In [21]:
gf.loc[34].statusHistory

[{'time': '2023-08-14T21:39:19.317000',
  'code': 'completed',
  'message': 'Tasking request has been completed.'},
 {'time': '2023-08-14T17:01:30.611000',
  'code': 'accepted',
  'message': 'Request can be completely satisfied during validity window.'},
 {'time': '2023-08-14T16:59:40.673000',
  'code': 'submitted',
  'message': 'Request approved and submitted for scheduling'},
 {'time': '2023-08-14T16:58:10.362000',
  'code': 'review',
  'message': 'Tasking request ready for review.'},
 {'time': '2023-08-14T16:57:54.083000',
  'code': 'received',
  'message': 'Request created'}]

In [22]:
def get_collect_history(row):
    collects = client.get_collects_for_task(row.taskingrequestId)
    c = collects[0] #always only 1?
    df = pd.DataFrame(c['collectStatusHistory'])
    return df

test = get_collect_history(gf.loc[34])
test

,time,code,message
0,2023-08-22T12:52:39.720Z,delivered,The products have been processed and delivered
1,2023-08-16T03:55:44.087Z,collection_anomaly,"A collection anomaly has occurred, if possible..."
2,2023-08-16T03:55:36.266Z,delivered,The products have been processed and delivered
3,2023-08-16T01:16:08.512Z,collection_anomaly,"A collection anomaly has occurred, if possible..."
4,2023-08-15T15:56:15.143Z,delivered,The products have been processed and delivered
5,2023-08-15T15:56:10.939Z,qa,Products awaiting QA before delivery
6,2023-08-15T15:55:50.646Z,delivered,The products have been processed and delivered
7,2023-08-15T15:55:46.144Z,qa,Products awaiting QA before delivery
8,2023-08-15T14:38:33.029Z,collection_anomaly,"A collection anomaly has occurred, if possible..."
9,2023-08-14T21:39:14.009Z,delivered,The products have been processed and delivered


In [23]:
'collection_anomaly' in test.code.values

True

In [24]:
def check_collection_anomaly(row):
    df = get_collect_history(row)
    check = 'collection_anomaly' in df.code.values
    return check

In [25]:
check_collection_anomaly(gf.loc[34])

True

In [26]:
gf['collection_anomaly'] = gf.apply(check_collection_anomaly, axis=1)

In [27]:
cols = ['collection_anomaly','acquisition2delivery','submission2windowOpen','time2deliver', 'windowDuration','taskingrequestName','taskingrequestDescription']
gf[cols]

,collection_anomaly,acquisition2delivery,submission2windowOpen,time2deliver,windowDuration,taskingrequestName,taskingrequestDescription
0,False,1.412717,-3.449882,11.039746,86401.000,Paradise: subdaily-9,Paradise subdaily test
2,False,2.084954,-3.310301,11.395361,86401.000,Paradise: subdaily-10,Paradise subdaily test
7,False,3.436076,10.587220,26.779765,86401.000,Paradise: subdaily-6,Paradise subdaily test
10,False,1.303516,-0.022800,18.334758,86400.000,Paradise Snotel 45 Retask Oct2,snotel station spotlight (ID: b3cc101b-ebe0-4d...
11,False,8.238487,47.626122,73.809439,86400.000,Baker Field Work: 3,Easton glacier
12,False,2.565781,23.627516,43.701188,86400.000,Baker Field Work: 2,Easton glacier
13,True,2.780717,-0.047348,10.944945,86400.000,Baker Field Work: 1,Easton glacier
14,False,20.261465,1.323689,47.211687,86400.000,Rainier 53 NE Strip Retask,"camp muir, NE strike (ID: 033fd86f-9271-42d9-9..."
15,False,2.316606,-0.039153,12.848820,86400.000,Paradise Snotel 45 Retask,snotel station spotlight (ID: b3cc101b-ebe0-4d...
18,False,2.042424,-0.037969,23.174859,86400.000,Rainier 53 NE Strip,"camp muir, NE strike"


In [28]:
print('Submission to delivery (hrs)')
gf.apply(time2delivery, axis=1).describe().apply('{:,.1f}'.format)

Submission to delivery (hrs)


count    13.0
mean     24.0
std      19.9
min       4.7
25%      11.0
50%      18.3
75%      26.8
max      73.8
dtype: object

In [29]:
# Drop initial task requests with unnecessarily large windows
print('Acquisition to delivery (hrs)')
gf[gf.windowDuration <= 86401].acquisition2delivery.describe().apply('{:,.1f}'.format)

Acquisition to delivery (hrs)


count    11.0
mean      4.3
std       5.7
min       0.6
25%       1.7
50%       2.3
75%       3.1
max      20.3
Name: acquisition2delivery, dtype: object